# Comparison of the performance of the Circuit-constraint

There are multiple ways to model the Traveling Salesman Problem (TSP) in CP-SAT.
In this notebook, we compare the performance of some.

In [1]:
TIMELIMIT_S = 10
STEP_SIZE = 10

## Generate random instances for a benchmark

To compare the performance of the different models, we need to generate random instances.
Because purely random graphs are not very interesting, we use points in the plane and the distance between them as edge weights.
This gives the instances a more realistic structure.
Without structure, solvers like CP-SAT and Gurobi cannot be smart and the comparison would be boring.

In [2]:
import networkx as nx
import random
import itertools
import typing


# Generate a random graph with a specified number of nodes.
def generate_random_euclidean_graph(
    n: int,
) -> typing.Tuple[nx.Graph, typing.List[typing.Tuple[int, int]]]:
    """
    Generate a random graph with a specified number of nodes.
    The nodes will be integers 0, 1, ..., n-1.
    The graph will be a complete graph with Euclidean distance as edge weight.
    """
    points = [(random.randint(0, 10_000), random.randint(0, 10_000)) for _ in range(n)]
    G = nx.Graph()
    G.add_nodes_from(range(n))
    # Complete graph with Euclidean distance as edge weight.
    for i, j in itertools.combinations(range(n), 2):
        p = points[i]
        q = points[j]
        dist = round((p[0] - q[0]) ** 2 + (p[1] - q[1]) ** 2)
        G.add_edge(i, j, weight=dist)
    return G, points

### Models

### Model 1: AddCircuit-constraint

In [3]:
from ortools.sat.python import cp_model


def solve_with_add_circuit(G: nx.Graph) -> typing.Tuple[bool, float]:
    """Solve TSP with add_circuit."""
    model = cp_model.CpModel()

    # Variables
    edge_vars = dict()
    for u, v in G.edges:
        edge_vars[u, v] = model.new_bool_var(f"edge_{u}_{v}")
        edge_vars[v, u] = model.new_bool_var(f"edge_{v}_{u}")

    # Constraints
    # Because the nodes in the graph a indices 0, 1, ..., n-1, we can use the
    # indices directly in the constraints. Otherwise, we would have to use
    # a mapping from the nodes to indices.
    circuit = [(u, v, x) for (u, v), x in edge_vars.items()]
    model.add_circuit(circuit)

    # Objective
    model.minimize(sum(x * G[u][v]["weight"] for (u, v), x in edge_vars.items()))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = TIMELIMIT_S
    # solver.parameters.log_search_progress = True
    status = solver.solve(model)
    return status == cp_model.OPTIMAL, solver.objective_value


assert solve_with_add_circuit(generate_random_euclidean_graph(10)[0])[
    0
], "Small graph should be solvable."

### Model 2: Miller Tucker Zemlin

In [4]:
def solve_with_miller_tucker_zemlin(G: nx.Graph) -> typing.Tuple[bool, float]:
    model = cp_model.CpModel()

    # Variables
    edge_vars = dict()
    for u, v in G.edges:
        edge_vars[u, v] = model.new_bool_var(f"edge_{u}_{v}")
        edge_vars[v, u] = model.new_bool_var(f"edge_{v}_{u}")

    depth_vars = dict()
    for u in G.nodes:
        depth_vars[u] = model.new_int_var(0, G.number_of_nodes() - 1, f"depth_{u}")

    # Constraints
    # Every node has exactly one incoming and one outgoing edge.
    for v in G.nodes:
        model.add_exactly_one([edge_vars[u, v] for u in G.neighbors(v)])
        model.add_exactly_one([edge_vars[v, u] for u in G.neighbors(v)])

    # Use depth variables to prohibit subtours.
    model.add(depth_vars[0] == 0)  # fix the root node to depth 0
    for v, w in G.edges:
        if w != 0:  # The root node is special.
            model.add(depth_vars[v] + 1 == depth_vars[w]).only_enforce_if(
                edge_vars[v, w]
            )
        if v != 0:
            model.add(depth_vars[w] + 1 == depth_vars[v]).only_enforce_if(
                edge_vars[w, v]
            )

    # Objective
    model.minimize(sum(x * G[u][v]["weight"] for (u, v), x in edge_vars.items()))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = TIMELIMIT_S
    # solver.parameters.log_search_progress = True
    status = solver.solve(model)
    return status == cp_model.OPTIMAL, solver.objective_value


assert solve_with_miller_tucker_zemlin(generate_random_euclidean_graph(10)[0])[
    0
], "Small graph should be solvable."

In [5]:
challengers = {
    "add_circuit": solve_with_add_circuit,
    "miller_tucker_zemlin": solve_with_miller_tucker_zemlin,
}
n = STEP_SIZE
while challengers:
    print(f"n={n}")
    G, points = generate_random_euclidean_graph(n)
    dropouts = []
    for name, solver in challengers.items():
        solved, obj = solver(G)
        if not solved:
            dropouts.append(name)
        else:
            print(f"  {name} solved it with objective {obj}.")
    for name in dropouts:
        print(f"  {name} dropped out.")
        del challengers[name]
    n += STEP_SIZE

n=10
  add_circuit solved it with objective 185396654.0.
  miller_tucker_zemlin solved it with objective 185396654.0.
n=20
  add_circuit solved it with objective 109773872.0.
  miller_tucker_zemlin solved it with objective 109773872.0.
n=30
  add_circuit solved it with objective 83156520.0.
  miller_tucker_zemlin solved it with objective 83156520.0.
n=40
  add_circuit solved it with objective 85448930.0.
  miller_tucker_zemlin solved it with objective 85448930.0.
n=50
  add_circuit solved it with objective 106851566.0.
  miller_tucker_zemlin dropped out.
n=60
  add_circuit solved it with objective 74616450.0.
n=70
  add_circuit solved it with objective 88725432.0.
n=80
  add_circuit solved it with objective 77799112.0.
n=90
  add_circuit dropped out.


### Model 3: Iterative Dantzig Model

Usually, you would use callbacks to add constraints dynamically.
However, this is not possible in the CP-SAT solver.
Because adding all constraints from the beginning is prohibitive, we add the constraints after each round, and solve the model again.

In [6]:
import time


class SubtourCallback(cp_model.CpSolverSolutionCallback):
    """
    Callback to detect subtours during the search.
    """

    def __init__(self, edge_vars: typing.Dict, early_abort=False):
        """
        edge_vars: a dictionary mapping edges to variables.
        early_abort: if True, the search will be aborted as soon as a subtour is detected. This is closer to lazy constraints, but may waste the previous work.
        """
        super().__init__()
        self.edge_vars = edge_vars
        self.subtours = []
        self.early_abort = early_abort

    def on_solution_callback(self):
        connected_components = list(
            nx.connected_components(
                nx.Graph(
                    (u, v) for (u, v), x in self.edge_vars.items() if self.value(x)
                )
            )
        )
        if len(connected_components) > 1:
            self.subtours += connected_components
            if self.early_abort:
                self.stop_search()

    def has_subtours(self):
        return len(self.subtours) > 0

    def reset(self):
        self.subtours = []


def solve_with_dantzig(G: nx.Graph) -> typing.Tuple[bool, float]:
    model = cp_model.CpModel()

    # Variables
    edge_vars = dict()
    for u, v in G.edges:
        # symmetric variables
        x = model.new_bool_var(f"edge_{u}_{v}")
        edge_vars[u, v] = x
        edge_vars[v, u] = x

    # Constraints
    # Every nodes has two incident edges.
    for v in G.nodes:
        model.add(sum(edge_vars[u, v] for u in G.neighbors(v)) == 2)

    # Objective
    tour_cost = sum(G.edges[u, v]["weight"] * edge_vars[(u, v)] for u, v in G.edges)
    model.minimize(tour_cost)

    solver = cp_model.CpSolver()

    start_time = time.time()

    def remaining_time():
        return TIMELIMIT_S - (time.time() - start_time)

    solver.parameters.max_time_in_seconds = remaining_time()
    callback = SubtourCallback(edge_vars, early_abort=False)
    # solver.parameters.log_search_progress = True
    status = solver.solve(model, callback)

    # The following part is more complex. Here we repeatedly add constraints
    # and solve the model again, until we find a solution without subtours.
    # TODO: Use the previous solutions as hints for the next solve.
    while callback.has_subtours():
        subtours = callback.subtours
        if len(subtours) > 1:
            for comp in subtours:
                outgoing_edges = sum(
                    edge_vars[u, v]
                    for u in comp
                    for v in G.neighbors(u)
                    if v not in comp
                )
                model.add(outgoing_edges >= 2)
            callback.reset()
            tour_cost = sum(
                G.edges[u, v]["weight"] * edge_vars[(u, v)] for u, v in G.edges
            )
            model.add(
                tour_cost >= int(solver.best_objective_bound)
            )  # help with lower bound
            solver.parameters.max_time_in_seconds = remaining_time()
            if remaining_time() <= 0:
                return False, solver.objective_value
            status = solver.solve(model, callback)
        else:
            break
    return status == cp_model.OPTIMAL, solver.objective_value


assert solve_with_dantzig(generate_random_euclidean_graph(20)[0])[
    0
], "Small graph should be solvable."

In [7]:
challengers = {
    "add_circuit": solve_with_add_circuit,
    "miller_tucker_zemlin": solve_with_miller_tucker_zemlin,
    "dantzig": solve_with_dantzig,
}
n = STEP_SIZE
while challengers:
    print(f"n={n}")
    G, points = generate_random_euclidean_graph(n)
    dropouts = []
    for name, solver in challengers.items():
        solved, obj = solver(G)
        if not solved:
            dropouts.append(name)
        else:
            print(f"  {name} solved it with objective {obj}.")
    for name in dropouts:
        print(f"  {name} dropped out.")
        del challengers[name]
    n += STEP_SIZE

n=10
  add_circuit solved it with objective 97715906.0.
  miller_tucker_zemlin solved it with objective 97715906.0.
  dantzig solved it with objective 97715906.0.
n=20
  add_circuit solved it with objective 86770784.0.
  miller_tucker_zemlin solved it with objective 86770784.0.
  dantzig solved it with objective 86770784.0.
n=30
  add_circuit solved it with objective 103530600.0.
  miller_tucker_zemlin solved it with objective 103530600.0.
  dantzig solved it with objective 103530600.0.
n=40
  add_circuit solved it with objective 76480910.0.
  miller_tucker_zemlin solved it with objective 76480910.0.
  dantzig solved it with objective 76480910.0.
n=50
  add_circuit solved it with objective 77015310.0.
  miller_tucker_zemlin solved it with objective 77015310.0.
  dantzig solved it with objective 77015310.0.
n=60
  add_circuit solved it with objective 88887270.0.
  dantzig solved it with objective 88887270.0.
  miller_tucker_zemlin dropped out.
n=70
  add_circuit solved it with objective

### Model 4: Using a classical MIP-Solver (Gurobi)

This is the classical approach to solve the TSP, as the linear programming relaxation is already a good approximation to the solution.

In [17]:
import gurobipy as gp


def solve_with_gurobi(G: nx.Graph) -> typing.Tuple[bool, float]:
    model = gp.Model()
    model.Params.LogToConsole = 0
    model.Params.TimeLimit = TIMELIMIT_S
    model.Params.lazyConstraints = 1

    # Variables
    edge_vars = dict()
    for u, v in G.edges:
        # symmetric variables
        x = model.addVar(vtype=gp.GRB.BINARY, name=f"edge_{u}_{v}")
        edge_vars[u, v] = x
        edge_vars[v, u] = x

    def gurobi_subtour_callback(model, where):
        if where == gp.GRB.Callback.MIPSOL:
            connected_components = list(
                nx.connected_components(
                    nx.Graph(
                        (u, v)
                        for (u, v), x in edge_vars.items()
                        if model.cbGetSolution(x) > 0.5
                    )
                )
            )
            if len(connected_components) > 1:
                for comp in connected_components:
                    outgoing_edges = sum(
                        edge_vars[u, v]
                        for u in comp
                        for v in G.neighbors(u)
                        if v not in comp
                    )
                    model.cbLazy(outgoing_edges >= 2)

    # Constraints
    # Every nodes has two incident edges.
    for v in G.nodes:
        model.addConstr(sum(edge_vars[u, v] for u in G.neighbors(v)) == 2)

    # Objective
    tour_cost = sum(G.edges[u, v]["weight"] * edge_vars[(u, v)] for u, v in G.edges)
    model.setObjective(tour_cost, gp.GRB.MINIMIZE)

    model.optimize(gurobi_subtour_callback)
    return model.status == gp.GRB.OPTIMAL, model.objVal


assert solve_with_gurobi(
    generate_random_euclidean_graph(20)[0]
), "Small graph should be solvable."

In [18]:
challengers = {
    "add_circuit": solve_with_add_circuit,
    "miller_tucker_zemlin": solve_with_miller_tucker_zemlin,
    "dantzig": solve_with_dantzig,
    "gurobi": solve_with_gurobi,
}
n = STEP_SIZE
while challengers:
    print(f"n={n}")
    G, points = generate_random_euclidean_graph(n)
    dropouts = []
    for name, solver in challengers.items():
        solved, obj = solver(G)
        if not solved:
            dropouts.append(name)
        else:
            print(f"  {name} solved it with objective {obj}.")
    for name in dropouts:
        print(f"  {name} dropped out.")
        del challengers[name]
    n += STEP_SIZE

n=10
  add_circuit solved it with objective 100129810.0.
  miller_tucker_zemlin solved it with objective 100129810.0.
  dantzig solved it with objective 100129810.0.
  gurobi solved it with objective 100129810.0.
n=20
  add_circuit solved it with objective 125047848.0.
  miller_tucker_zemlin solved it with objective 125047848.0.
  dantzig solved it with objective 125047848.0.
  gurobi solved it with objective 125047848.0.
n=30
  add_circuit solved it with objective 95776200.0.
  miller_tucker_zemlin solved it with objective 95776200.0.
  dantzig solved it with objective 95776200.0.
  gurobi solved it with objective 95776200.00000001.
n=40
  add_circuit solved it with objective 90770264.0.
  miller_tucker_zemlin solved it with objective 90770264.0.
  dantzig solved it with objective 90770264.0.
  gurobi solved it with objective 90770264.0.
n=50
  add_circuit solved it with objective 87023666.0.
  gurobi solved it with objective 87023666.0.
  miller_tucker_zemlin dropped out.
  dantzig d

## Benchmark

Now let us do a slightly more serious benchmark, by running each model multiple times and averaging the results.
This is still not a very good benchmark, because the dropout can skew the results.
It would be better to let each approach try to solve every instance, and then compare the results by looking at how many instances of each size could still be solved.
However, our approach here is more time efficient.

In [19]:
def run_dropout_benchmark():
    challengers = {
        "add_circuit": solve_with_add_circuit,
        "miller_tucker_zemlin": solve_with_miller_tucker_zemlin,
        "dantzig": solve_with_dantzig,
        "gurobi": solve_with_gurobi,
    }
    dropout_points = dict()
    n = STEP_SIZE
    while challengers:
        G, points = generate_random_euclidean_graph(n)
        dropouts = []
        for name, solver in challengers.items():
            solved, obj = solver(G)
            if not solved:
                dropouts.append(name)
        for name in dropouts:
            del challengers[name]
            dropout_points[name] = n
        n += STEP_SIZE
    return dropout_points


dropout_points = [run_dropout_benchmark() for _ in range(10)]
# compute the average
dropout_points = {
    name: sum(d[name] for d in dropout_points) / len(dropout_points)
    for name in dropout_points[0]
}
print(dropout_points)

{'dantzig': 48.0, 'miller_tucker_zemlin': 48.0, 'add_circuit': 110.0, 'gurobi': 225.0}
